In [ ]:
%matplotlib inline
import torch
from torch import nn
import sys
sys.path.append('..')
from tools import tool_pytorch_017 as ty

'''
创建一个1000个点的集合，从1开始，一直到1000，作为一个连续的序列
'''

T = 1000  # 总共产生1000个点
time = torch.arange(1, T + 1, dtype=torch.float32)
#在序列中加入噪声数据，具体是加入均值为0方差为0.2的数据
x = torch.sin(0.01 * time) + torch.normal(0, 0.2, (T,))
ty.plot(time, [x], 'time', 'x', xlim=[1, 1000], figsize=(6, 3))

In [ ]:
# 设置tau为4，也就是说以当前元素的前四个元素为学习依据
tau = 4
features = torch.zeros((T - tau, tau))
for i in range(tau):
    # 从x中取出下标数据作为特征集
    features[:, i] = x[i: T - tau + i]
# 取出tau后的一个数据，作为值
labels = x[tau:].reshape((-1, 1))


# 初始化网络权重的函数
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

# 一个简单的多层感知机
def get_net():
    net = nn.Sequential(nn.Linear(4, 10),
                        nn.ReLU(),
                        nn.Linear(10, 1))
    net.apply(init_weights)
    return net

# 平方损失。注意：MSELoss计算平方误差时不带系数1/2
loss = nn.MSELoss(reduction='none')

def train(net, train_iter, loss, epochs, lr):
    trainer = torch.optim.Adam(net.parameters(), lr)
    for epoch in range(epochs):
        for X, y in train_iter:
            trainer.zero_grad()
            l = loss(net(X), y)
            l.sum().backward()
            trainer.step()
        print(f'epoch {epoch + 1}, '
              f'loss: {ty.evaluate_loss(net, train_iter, loss):f}')

batch_size, n_train = 16, 600
# 只有前n_train个样本用于训练
train_iter = ty.load_array((features[:n_train], labels[:n_train]),
                            batch_size, is_train=True)
net = get_net()
train(net, train_iter, loss, 5, 0.01)

# 开始单步预测，也就是说每次只位移一个元素
# 我们训练的次数为600，也就是说我们单步预测只向前位移了604个元素
# 但604个元素以后的预测结果也是不错的
onestep_preds = net(features)
ty.plot([time, time[tau:]],
         [x.detach().numpy(), onestep_preds.detach().numpy()], 'time',
         'x', legend=['data', '1-step preds'], xlim=[1, 1000],
         figsize=(6, 3))

In [ ]:
# 进行多步预测，多步预测将从第604个元素起使用自己的数据来做
# 这里是初始化成0了
# 效果可想而知，预测结果接近-0.1
multistep_preds = torch.zeros(T)
multistep_preds[: n_train + tau] = x[: n_train + tau]
for i in range(n_train + tau, T):
    multistep_preds[i] = net(
        multistep_preds[i - tau:i].reshape((1, -1)))

ty.plot([time, time[tau:], time[n_train + tau:]],
         [x.detach().numpy(), onestep_preds.detach().numpy(),
          multistep_preds[n_train + tau:].detach().numpy()], 'time',
         'x', legend=['data', '1-step preds', 'multistep preds'],
         xlim=[1, 1000], figsize=(6, 3))

In [ ]:
# 通过对整个序列预测的计算， 让我们更仔细地看一下每次位移1、4、16、64步预测的困难。
max_steps = 64

features = torch.zeros((T - tau - max_steps + 1, tau + max_steps))
# 列i（i<tau）是来自x的观测，其时间步从（i）到（i+T-tau-max_steps+1）
for i in range(tau):
    features[:, i] = x[i: i + T - tau - max_steps + 1]

# 列i（i>=tau）是来自（i-tau+1）步的预测，其时间步从（i）到（i+T-tau-max_steps+1）
for i in range(tau, tau + max_steps):
    features[:, i] = net(features[:, i - tau:i]).reshape(-1)

steps = (1, 4, 16, 64)
ty.plot([time[tau + i - 1: T - max_steps + i] for i in steps],
         [features[:, (tau + i - 1)].detach().numpy() for i in steps], 'time', 'x',
         legend=[f'{i}-step preds' for i in steps], xlim=[5, 1000],
         figsize=(6, 3))